### 필요한 라이브러리 import

In [1]:
import pandas as pd
import os
import requests
import sys

### KAKAO API KEY 값 관리

In [2]:
api_key_url = '../API KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
open_api_key_file['사이트'].value_counts()

get_api_key = open_api_key_file[open_api_key_file['사이트'] == '카카오 REST'].values[0][1]  

In [3]:
# KAKAO API KEY 값 
KAKAO_API_KEY = get_api_key  

### 파일을 불러올 폴더의 위치, 저장할 폴더의 위치 설정

In [4]:
folder_name_1 = './data'
folder_name_2 = './make_file'

save_file_folder = f'{folder_name_2}/'
sava_file_encoding = 'utf-8'

### 파일을 저장할 위치 확인

In [5]:
folder_1_check = os.path.isdir(folder_name_1)

if not folder_1_check:
    os.makedirs(folder_name_1)  # folder_name_1이 없으면 생성
    print(f"'{folder_name_1}' 폴더가 생성되었습니다.")

folder_2_check = os.path.isdir(save_file_folder)
if not folder_2_check:
    os.makedirs(save_file_folder)  # folder_name_2가 없으면 생성
    print(f"'{save_file_folder}' 폴더가 생성되었습니다.")


'./make_file/' 폴더가 생성되었습니다.


### 불러올 파일들의 폴더 위치 지정

In [6]:
temp_school_url = f'{folder_name_1}/서울시_광진구_학교_기본정보.csv'
temp_children_file_url =  f'{folder_name_1}/서울시_광진구_지역아동센터_기본정보.csv'
temp_childcare_file_url = f'{folder_name_1}/서울시_광진구_어린이집_정보표준_데이터.csv'
temp_welfare_file_url = f'{folder_name_1}/서울시_광진구_복지센터_정보표준_데이터.csv'

school_file_check = os.path.isfile(temp_school_url)
children_file_check = os.path.isfile(temp_children_file_url) 
childcare_file_check = os.path.isfile(temp_childcare_file_url)
welfare_care_file_check = os.path.isfile(temp_welfare_file_url)

def sys_exit(err_file) :
    print(f'{err_file} 파일이 없습니다....')
    print(f'파일이 없으므로 시스템을 종료합니다.')
    sys.exit()

if school_file_check is False :
    sys_exit(temp_school_url)
    
if children_file_check is False :
    sys_exit(temp_children_file_url)

if childcare_file_check is False :
    sys_exit(temp_childcare_file_url)

if welfare_care_file_check is False :
    sys_exit(temp_seniorcare_file_url)
    
if (school_file_check and school_file_check and childcare_file_check and welfare_care_file_check) :
    print(f'필요한 파일이 모두 있습니다.')
    
school_url = temp_school_url
children_file_url = temp_children_file_url
childcare_file_url = temp_childcare_file_url
welfare_file_url = temp_welfare_file_url

필요한 파일이 모두 있습니다.


### 통합 파일로 저장하기 위한 배열

In [7]:
# 통합 파일로 저장할 배열
all_list_classification = []
all_list_name = []
all_list_location_x_y = []
all_list_location_x = []
all_list_location_y = [] 

### 학교 목록 공공데이터를 불러와서 처리 시작

In [8]:
school_data = pd.read_csv(school_url, encoding = 'euc-kr')

In [9]:
# school_data.head(5)

In [10]:
# school_data.info()

In [11]:
# school_data.isnull().sum()

In [12]:
school_data = school_data.drop_duplicates(subset='학교명')
# school_data

In [13]:
school_data['학교종류명'].unique()

array(['각종학교(중)', '중학교', '초등학교', '외국인학교', '고등학교', '특수학교'], dtype=object)

In [14]:
change_school_classification = {
    '초등학교' : 1,
    '각종학교(중)' : 2,
    '중학교' : 2,
    '고등학교' : 3,
    '외국인학교' : 4, # 외국인학교
    '특수학교' : 5 # 특수학교
}

In [15]:
school_data['학교종류명'] = school_data['학교종류명'].map(change_school_classification)

In [16]:
# print(f'학교 종류 : {school_data['학교종류명'].unique()}')
# print(f'학 학교별 수 : {school_data['학교종류명'].value_counts()}')

In [17]:
# 초 중 고 여부. ['학교종류명']
#     '초등학교' : 1,
#     '각종학교(중)' : 2,
#     '중학교' : 2,
#     '고등학교' : 3,
#     '외국인학교' : 4,
#     '특수학교' : 5 
# school_classification = []

# # 학교 이름
# school_name = [] 

In [18]:
school_name = school_data['학교명']
school_classification = school_data['학교종류명']

### 학교 정보 저장을 위한 배열

In [19]:
get_school_name = []
get_school_classification = []
get_school_location_x_y = []
get_school_location_x = []
get_school_location_y = []

### kakao api를 활용하여 시설명 -> 위도, 경도 값으로 변환

In [ ]:
def kakao_api(facilities, district="광진구"):
    query = f"{district} {facilities}"
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": query}
    
    response = requests.get(url, headers=headers, params=params)
    result = response.json()  
    
    for item in result.get("documents", []):
        # 주소에서 district(광진구)가 포함된 데이터만 필터링
        if district in item["address_name"]:  
            lat = float(item["y"])  # 위도
            lon = float(item["x"])  # 경도
            return lat, lon  
         
    return None, None  

### 학교 수 만큼 반복문을 돌려서 위도,경도값 변환

In [21]:

for i in range(len(school_name)):
    search_school_name = school_name.iloc[i]  
    search_school_classification = school_classification.iloc[i] 
    
    latitude, longitude = kakao_api(search_school_name)

    search_school_location_x_y = f'{latitude}, {longitude}' 

    get_school_name.append(search_school_name)
    get_school_classification.append(search_school_classification)
    get_school_location_x_y.append(search_school_location_x_y)
    get_school_location_x.append(latitude)
    get_school_location_y.append(longitude)
    


### 학교 위치정보 저장을 위한 데이터프레임화

In [22]:
school_df = pd.DataFrame({
    '학교구분' : get_school_classification, 
    '학교명' : get_school_name, 
    '학교_위치' : get_school_location_x_y,
    '위도' : get_school_location_x,
    '경도' : get_school_location_y
})

### 아동센터 파일 처리

In [23]:
children_data = pd.read_csv(children_file_url, encoding = 'euc-kr')
# children_data.head(2)

In [24]:
children_center_name = children_data['센터명']
children_center_x_y_location = []
children_center_x = []
children_center_y = []
children_center_classification = []

temp_x = children_data['위도']
temp_y = children_data['경도']

for i in range(len(children_center_name)) :
    temp_x_y = f'{temp_x[i]}, {temp_y[i]}'
    children_center_x_y_location.append(temp_x_y)
    children_center_x.append(temp_x[i])
    children_center_y.append(temp_y[i])
    children_center_classification.append('아동센터') 

### 아동센터 위치정보 저장을 위한 데이터프레임화

In [25]:
#개별 데이터 저장
children_df = pd.DataFrame({
    '구분' : children_center_classification, 
    '시설명' : children_center_name, 
    '시설_위치' : children_center_x_y_location
})


### 어린이집 파일 처리

In [26]:
# children_care_data = pd.read_csv(childcare_file_url, encoding = 'euc-kr')
temp_children_care_data = pd.read_csv(childcare_file_url, encoding='euc-kr', on_bad_lines='skip', 
                            na_values=['', 'NaN', 'null', 'NULL', 'None', 'none', '0'])

# children_care_data.head(2)

In [27]:
get_children_care_name = []
get_children_care_classification = []
get_children_care_location_x_y = []
get_children_care_location_x = []
get_children_care_location_y = []

In [28]:
temp_children_care_data['운영현황'].unique()

array(['정상', '폐지', '재개', '휴지'], dtype=object)

In [29]:
children_care_data = temp_children_care_data[temp_children_care_data['운영현황'].isin(['재개', '정상'])]
children_care_data = children_care_data.reset_index(drop=True)
children_care_data.index = children_care_data.index + 1

In [30]:
children_care_data['운영현황'].unique()

array(['정상', '재개'], dtype=object)

In [31]:
children_care_data['어린이집명'] = children_care_data['어린이집명'].str.replace('구립 ', '', regex=True)
children_care_data['어린이집명'] = children_care_data['어린이집명'].str.replace('구립', '', regex=True)

In [32]:
children_care_name = children_care_data['어린이집명']

for i in range(len(children_care_name)):
    search_children_care_name = children_care_name.iloc[i] 
    search_children_care_classification = '어린이집'

    temp_latitude, temp_longitude = kakao_api(search_children_care_name)

    search_children_care_location_x_y = f'{temp_latitude}, {temp_longitude}' 

    if((temp_latitude is not None) and (temp_longitude is not None)) :
        get_children_care_name.append(search_children_care_name)
        get_children_care_classification.append(search_children_care_classification)
        get_children_care_location_x_y.append(search_children_care_location_x_y)
        get_children_care_location_x.append(temp_latitude)
        get_children_care_location_y.append(temp_longitude)     


### 어린이집을 파일로 저장하기 위한 프레임워크화

In [33]:
children_care_df = pd.DataFrame({
    '구분' : get_children_care_classification, 
    '시설명' : get_children_care_name, 
    '위치' : get_children_care_location_x_y,
    '위도' : get_children_care_location_x,
    '경도' : get_children_care_location_y
})

### 복지센터 파일을 처리

In [34]:
temp_welfare_care_file = pd.read_csv(welfare_file_url, encoding='utf-8')

# welfare_care_file.head(2)

In [35]:
len(temp_welfare_care_file['시설명'])

108

In [36]:
welfare_care_file = temp_welfare_care_file.drop_duplicates(subset='시설명') 

In [37]:
len(welfare_care_file['시설명'])

99

In [38]:
get_welfare_center_name = []
get_welfare_center_x_y_location = []
get_welfare_center_x = []
get_welfare_center_y = []
get_welfare_center_classification = []

In [39]:
welfare_center_name = welfare_care_file['시설명']

for i in range(len(welfare_center_name)):
    search_welfare_center_name = welfare_center_name.iloc[i] 
    search_welfare_center_classification = '복지센터'

    temp_latitude, temp_longitude = kakao_api(search_welfare_center_name)

    search_welfare_center_location_x_y = f'{temp_latitude}, {temp_longitude}' 

    if((temp_latitude is not None) and (temp_longitude is not None)) :
        get_welfare_center_name.append(search_welfare_center_name)
        get_welfare_center_classification.append(search_welfare_center_classification)
        get_welfare_center_x_y_location.append(search_welfare_center_location_x_y)
        get_welfare_center_x.append(temp_latitude)
        get_welfare_center_y.append(temp_longitude)      

### 복지센터 파일로 저장하기 위한 프레임워크화

In [40]:
welfare_center_df = pd.DataFrame({
    '구분' : get_welfare_center_classification, 
    '시설명' : get_welfare_center_name, 
    '위치' : get_welfare_center_x_y_location,
    '위도' : get_welfare_center_x,
    '경도' : get_welfare_center_y
})

### 학교 위치를 csv 파일로 저장

In [41]:
school_df.to_csv(f'{save_file_folder}/광진구_학교_위치.csv', encoding = sava_file_encoding)

### 아동센터 위치를 csv 파일로 저장

In [42]:
children_df.to_csv(f'{save_file_folder}/광진구_아동센터_위치.csv', encoding = sava_file_encoding)

### 어린이집 위치를 csv 파일로 저장

In [43]:
children_care_df.to_csv(f'{save_file_folder}/광진구_어린이집_위치.csv', encoding = sava_file_encoding)

### 복지센터 위치를 csv 파일로 저장

In [44]:
welfare_center_df.to_csv(f'{save_file_folder}/광진구_복지센터_위치.csv', encoding = sava_file_encoding)

### 통합 파일로 저장하기 위한 데이터값 저장

In [45]:
# 통합 파일에 학교 정보를 저장
for i in range(len(get_school_name)) :
    all_list_classification.append('학교')
    all_list_location_x_y.append(get_school_location_x_y[i])
    all_list_name.append(get_school_name[i])
    all_list_location_x.append(get_school_location_x[i])
    all_list_location_y.append(get_school_location_y[i])

In [46]:
# 통합 파일에 아동센터 정보를 저장
for i in range(len(children_center_name)) :
    all_list_classification.append('아동센터')
    all_list_name.append(children_center_name[i])
    all_list_location_x_y.append(children_center_x_y_location[i])
    all_list_location_x.append(children_center_x[i]) 
    all_list_location_y.append(children_center_y[i]) 
    

In [47]:
# 통합 파일에 어린이집 정보를 저장
for i in range(len(get_children_care_name)) :
    all_list_classification.append('어린이집')
    all_list_name.append(get_children_care_name[i])
    all_list_location_x_y.append(get_children_care_location_x_y[i])
    all_list_location_x.append(get_children_care_location_x[i]) 
    all_list_location_y.append(get_children_care_location_y[i]) 
    

In [48]:
# 통합 파일에 복지센터 정보를 저장
for i in range(len(get_welfare_center_name)) :
    all_list_classification.append('복지센터')
    all_list_name.append(get_welfare_center_name[i])
    all_list_location_x_y.append(get_welfare_center_x_y_location[i])
    all_list_location_x.append(get_welfare_center_x[i]) 
    all_list_location_y.append(get_welfare_center_y[i]) 

### 통합 데이터 파일을 csv 파일로 저장

In [49]:
# 통합 데이터 저장

all_list_df = pd.DataFrame({
    '시설_구분' : all_list_classification,
    '시설_이름' : all_list_name,
    '시설_위치' : all_list_location_x_y,
    '위도' : all_list_location_x,
    '경도' : all_list_location_y
})

all_list_df.to_csv(f'{folder_name_2}/광진구_모든_시설의_위치.csv', encoding = sava_file_encoding)

In [50]:
print(f'시스템 종료')

시스템 종료
